In [ ]:
import numpy as np
import os

In [ ]:
def fetch(url):
    import requests, gzip, os, hashlib, pathlib, numpy
    
    # fp = pathlib.Path.joinpath("/tmp" , hashlib.md5(url.encode('utf-8')).hexdigest() )
    fp = "C:/Users/Prarabdha/Desktop/TensorLabs.ai/tmp/" + hashlib.md5(url.encode('utf-8')).hexdigest()
    

    if os.path.isfile(fp):
        with open(fp , "rb") as f:
            data = f.read()
    else:
        with open(fp , "wb") as f:
            data = requests.get(url).content
            f.write(data) 
    return numpy.frombuffer(gzip.decompress(data) , dtype=np.uint8).copy()

X_train = fetch('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz')[0x10:].reshape((-1 , 28 , 28))
Y_train = fetch('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz')[8:]
X_test = fetch('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz')[0x10:].reshape((-1 , 28 , 28))
Y_test = fetch('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz')[8:]

In [ ]:
import torch
import torch.nn as nn
class PopNet(torch.nn.Module):
    def __init__(self):
        super(PopNet , self).__init__()
        self.l1 = nn.Linear(784 , 128)
        self.act = nn.ReLU()
        self.l2 = nn.Linear(128 , 10)

    def forward(self , x):
        x = self.l1(x)
        x = self.act(x)
        x = self.l2(x)
        return x
    
model = PopNet()
model(torch.tensor(X_train[0:10].reshape((-1 , 28*28)) , dtype=torch.uint8).float())

In [ ]:
from tqdm import tqdm
batch_size = 32
loss_function = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters())
losses , accuracies = [] , []
for i in tqdm(range(1000)):
    samp = np.random.randint(0 , X_train.shape[0] , size = (batch_size))
    X = torch.tensor(X_train[samp].reshape((-1 , 28*28)) , dtype=torch.uint8).float()
    Y = torch.tensor(Y_train[samp] , dtype=torch.uint8).long()

    optim.zero_grad()
    out = model(X)
    cat = torch.argmax(out , dim=1)
    accuracy = (cat == Y).float().mean()

    loss = loss_function(out , Y)
    loss.backward()
    optim.step()
    loss , accuracy = loss.item() , accuracy.item()
    losses.append(loss)
    accuracies.append(accuracy)
print(loss , accuracy)

In [ ]:
import matplotlib.pyplot as plt

# a = plt.subplot((1,2))
plt.plot(losses)
plt.plot(accuracies)

In [ ]:
for i in tqdm(range(X_test.shape[0])):

    x = torch.tensor(X_test[i].reshape((-1 , 28*28)) , dtype=torch.uint8).float()
    y = torch.tensor(Y_test[i] , dtype=torch.uint8).long()
    out = model(x)
    cat = torch.argmax(out , dim=1)
    test_accuracy = (out == y).float().mean()

print(test_accuracy)

In [ ]:
l1 = np.zeros((28*28 , 128) , dtype=np.float32)
l2 = np.zeros((128 , 10) , dtype=np.float32)

In [ ]:
import numpy
l1[ : ] = model.l1.weight.detach().numpy().transpose()
l2[:] = model.l2.weight.detach().numpy().transpose()

In [ ]:
def forward(x):
    x = x.dot(l1)
    x = np.maximum(x , 0)
    x = x.dot(l2)
    return x

print(forward(X_test.reshape((-1 , 28*28))))

In [ ]:
Y_test_preds_out = forward(X_test.reshape((-1 , 28*28)))
Y_test_preds = np.argmax(Y_test_preds_out , axis=1)
(Y_test_preds == Y_test).mean()

In [ ]:
ret = -Y_test_preds_out[range(Y_test_preds_out.shape[0]) , Y_test] + np.log(np.exp(Y_test_preds_out).sum(axis=1))

In [ ]:

G=32
grid = sorted(list(zip(ret , range(ret.shape[0]))) , reverse=False)[0:G*G]
x_bad = X_test[[x[1] for x in grid]]
plt.figure(figsize=(12,12))
plt.imshow(np.concatenate(x_bad.reshape((G , 28*G , 28)) , axis=1))

In [ ]:
samp = [0,1]

out = np.zeros((len(samp),10) , np.float32)
out[range(out.shape[0]) , Y_test[samp]] = 1

def logsumexp(x):
    c = x.max(axis=1)
    return c+np.log(np.exp(x-c.reshape((-1,1))).sum(axis=1))

def forward_backward(x,y):
    out = np.zeros((len(y),10) , np.float32)
    out[range(out.shape[0]) , y] = 1
    #forward pass
    # x = X_test[samp].reshape((-1 , 28*28))
    x_l1 = x.dot(l1)
    x_relu = np.maximum(x_l1 , 0)
    x_l2 = x_relu.dot(l2)
    x_lsm = x_l2 - np.log(np.exp(x_l2).sum(axis=1)).reshape((-1,1))
    x_loss = (-out * x_lsm).mean(axis=1)

    #backward pass

    d_out = -out/len(y)

    dx_lsm = d_out - np.exp(x_lsm)*d_out.sum(axis=1).reshape((-1,1))

    d_l2 = x_relu.T.dot(dx_lsm)
    dx_relu = dx_lsm.dot(l2.T)

    dx_l1 = (x_relu > 0).astype(np.float32) * dx_relu

    d_l1 = x.T.dot(dx_l1)

    # plt.imshow(d_l1.T)
    # plt.figure(figsize=(12,12))
    # plt.imshow(d_l2.T)
    return x_loss , x_l2 , d_l1 , d_l2

samp= range(32)
forward_backward(X_test[samp].reshape((-1 , 28*28)) , Y_test[samp])


In [ ]:
#numpyb training

def kaiming(m,h):
    ret = np.random.uniform(-1./np.sqrt(m*h) , 1./np.sqrt(m*h) , size = (m,h))
    return ret.astype(np.float32)

#init
lr = 0.001
l1 = kaiming(*l1.shape)
l2 = kaiming(*l2.shape)

BS = 128
losses , accuracies = [] , []
for i in tqdm(range(1000)):
    samp = np.random.randint(0 , X_train.shape[0] , size=(BS))
    X = X_train[samp].reshape((-1 , 28*28))
    Y = Y_train[samp] 
    x_loss , x_l2 , d_l1 , d_l2 = forward_backward(X,Y)

    l1 = l1 - lr*d_l1
    l2 = l2 - lr*d_l2
    cat = np.argmax(x_l2 , axis=1)
    accuracy = (cat==Y).mean()

    loss = x_loss.mean()
    losses.append(loss)
    accuracies.append(accuracy)

print(loss , accuracy)
plt.plot(losses)
plt.plot(accuracies)